In [1]:
import pandas as pd
import glob
from datetime import datetime,date
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymongo
import csv
import json
import re
import os
import psycopg2

# Connecting to Posgres

In [ ]:
#Connecting to Postgres
hostname = 'covid19.clksoczr9us2.us-east-1.rds.amazonaws.com'
port_id = 5432
username = 'postgres'
database = 'postgres'
pwd = 'User1234' 

conn = psycopg2.connect(
        host = hostname,
        dbname = database,
        user = username,
        password = pwd, 
        port = port_id)
print("database opened successfully")




# Downloading from COVID19_data

In [11]:
#Downloading the data from Postgres
df = pd.read_sql('SELECT * FROM covid19_data', conn)
df.head()

,city,province_state,country_region,lat,long,confirmed_cases,cases_per_day,last_update,people_fully_vaccinated,coordinates
0,Wayne,Illinois,US,38.429554,-88.425865,2155.0,NaN,2021-08-23,175676240,"[38.42955355, -88.42586459]"
1,White,Illinois,US,38.087062,-88.176575,2056.0,NaN,2021-08-23,175676240,"[38.08706198, -88.17657461]"
2,Whiteside,Illinois,US,41.755097,-89.915561,7414.0,NaN,2021-08-23,175676240,"[41.75509713, -89.91556058]"
3,Will,Illinois,US,41.446193,-87.978627,81616.0,NaN,2021-08-23,175676240,"[41.44619267, -87.97862712]"
4,Williamson,Illinois,US,37.730335,-88.929940,9461.0,NaN,2021-08-23,175676240,"[37.73033511, -88.92994027]"


# Create New Table for Machine Learning Data

In [ ]:
#Connecting to Postgres
hostname = 'covid19.clksoczr9us2.us-east-1.rds.amazonaws.com'
port_id = 5432
username = 'postgres'
database = 'postgres'
pwd = 'User1234' 

conn = psycopg2.connect(
        host = hostname,
        dbname = database,
        user = username,
        password = pwd, 
        port = port_id)
print("database opened successfully")


In [ ]:
#merged_df.dtypes

In [5]:
replacements = {
    'object':'varchar',
    'float64':'decimal',
    'datetime64[ns]':'date',
    'int64':'int'
}

In [6]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'city varchar, province_state varchar, country_region varchar, lat decimal, long decimal, confirmed_cases decimal, cases_per_day decimal, last_update varchar, people_fully_vaccinated int, coordinates varchar'

In [7]:
conn = psycopg2.connect(
        host = hostname,
        dbname = database,
        user = username,
        password = pwd, 
        port = port_id)
print("database opened successfully")

cursor = conn.cursor()

cursor.execute("Create Table covid19_data \
(City varchar, Province_State varchar, Country_Region varchar, Lat decimal, Long decimal, Confirmed_Cases decimal, Cases_Per_Day decimal, Last_Update date, people_fully_vaccinated int, Coordinates varchar)")



database opened successfully


# Save the new Merged_df to CSV

In [8]:
#Save to CSV file 
df.to_csv('COVID_19_dataV8_Confirmed_cases.csv', header=df.columns, index=False, encoding='utf-8')

my_file = open('COVID_19_dataV8_Confirmed_cases.csv')
print('file opened in memory')

file opened in memory


In [9]:
#upload to Postgres
SQL_STATEMENT = """
COPY covid19_data FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""

cursor.copy_expert(sql=SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [10]:
cursor.execute("grant select on table covid19_data to public")
conn.commit()
cursor.close()
print('covid19 imported to DB completed')

covid19_data imported to DB completed


In [4]:
# #Add the last two columns after coordinates

# cursor = conn.cursor()

# cursor.execute('''
#                 Create Table covid19_data(
#                     City VARCHAR,
#                     Province_State VARCHAR,
#                     Country_Region VARCHAR,
#                     Lat DECIMAl (10,8),
#                     Long DECIMAL (11,8),
#                     Confirmed_Cases Decimal (9,1),
#                     Cases_Per_day Decimal (9,1),
#                     Last_Update DATE,
#                     people_fully_vaccinated INT,
#                     Coordinates VARCHAR
#                     Real Values Decimal (9,1)
#                     Predicted Values Decimal (9,1);
#                 ''')